In [1]:
%load_ext autoreload
%autoreload 2

## Setup Dask

In [2]:
import dask
from dask.distributed import Client
from dask_jobqueue import SLURMCluster

dask.config.set(**{'array.slicing.split_large_chunks': False})

portdash = 10062

cluster = SLURMCluster(
    queue="batch",
    cores=8,
    processes=1,
    account="gfdl_o",
    memory="48GB",
    walltime="08:00:00",
    local_directory="$TMPDIR",
    death_timeout=240,
    scheduler_options={"dashboard_address":f":{portdash}"},
    job_extra_directives=["--exclude=pp[008-010],pp[013-075]"],
    job_name="mhw-metrics"
)

client = Client(cluster)

In [3]:
client

Connection method: Cluster object,Cluster type: dask_jobqueue.SLURMCluster
Dashboard: http://140.208.147.197:10062/status,
Dashboard: http://140.208.147.197:10062/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://140.208.147.197:33283,Workers: 0
Dashboard: http://140.208.147.197:10062/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [24]:
cluster.scale(jobs=0)

In [ ]:
cluster.close()
client.close()

## Import Modules

In [5]:
import xarray as xr
import numpy as np
import dask.array as da

import util
import events
import mhwmetrics
import annualcycle

In [6]:
%%time
ds = xr.open_zarr("/xtmp/Xinru.Li/work/cm4_thetao_0151_0250_fast_detrend", use_cftime=True)
temp_chunk = util.load_chunk(ds,times=(None,None),xrange=(500,520),yrange=(700,720))
temp_chunk

CPU times: user 101 ms, sys: 39.5 ms, total: 140 ms
Wall time: 513 ms


<xarray.DataArray 'thetao' (time: 36500, z2000_l: 16, yh: 20, xh: 20)>
dask.array<getitem, shape=(36500, 16, 20, 20), dtype=float32, chunksize=(36500, 16, 20, 20), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) object 0151-01-01 12:00:00 ... 0250-12-31 12:00:00
  * xh       (xh) float64 -175.6 -175.4 -175.1 -174.9 ... -171.4 -171.1 -170.9
  * yh       (yh) float64 44.11 44.29 44.47 44.65 ... 46.91 47.08 47.25 47.42
  * z2000_l  (z2000_l) float64 2.5 10.0 20.0 32.5 ... 400.0 500.0 600.0 700.0
Attributes:
    cell_measures:  volume: volcello area: areacello
    cell_methods:   area:mean z2000_l:mean yh:mean xh:mean time: mean
    long_name:      Sea Water Potential Temperature
    standard_name:  sea_water_potential_temperature
    time_avg_info:  average_T1,average_T2,average_DT
    units:          degC

In [16]:
%%time
def process_chunk(temp_chunk):
    temp_chunk = temp_chunk.load()
    mean, thresh = events.threshold_and_climo(temp_chunk)

    def _process_sub_chunk(temp_chunk, thresh, trange):
        sub_arr = temp_chunk.sel(time=slice(*trange))
        exceedences, thresh_tiled = events.calculate_execeedences(sub_arr, thresh)
        event_index, num_events = events.tag_events(exceedences)
        return mhwmetrics.calc_metrics(event_index, event_index, thresh_tiled)

    start = 151
    years = list(np.arange(start,start+100))
    subchunks = [(str(x).zfill(4)+"-01-01",str(x+29).zfill(4)+"-12-31") for x in years if x+29 < start+100]
    subchunks = subchunks[37:]
    
    dsmetrics = [_process_sub_chunk(temp_chunk, thresh, x) for x in subchunks]
    dsmetrics = xr.concat(dsmetrics, "window")
    dsmetrics = dsmetrics.assign_coords({"window":dsmetrics.window})

    try:
        assert "clim_freq" in dsmetrics.variables.keys()
    except Exception as exc:
        print(temp_chunk)
        print(dsmetrics)
        raise exc

    return dsmetrics

test_ds = process_chunk(temp_chunk)

Using Cython-accelerated percentile calculation
Using Cython-accelerated percentile calculation
CPU times: user 5min 31s, sys: 46.9 s, total: 6min 18s
Wall time: 5min 56s


In [17]:
test_ds

<xarray.Dataset>
Dimensions:      (xh: 20, yh: 20, z2000_l: 16, window: 34)
Coordinates:
  * xh           (xh) float64 -175.6 -175.4 -175.1 ... -171.4 -171.1 -170.9
  * yh           (yh) float64 44.11 44.29 44.47 44.65 ... 47.08 47.25 47.42
  * z2000_l      (z2000_l) float64 2.5 10.0 20.0 32.5 ... 500.0 600.0 700.0
  * window       (window) int64 0 1 2 3 4 5 6 7 8 ... 25 26 27 28 29 30 31 32 33
Data variables:
    clim_freq    (window, z2000_l, yh, xh) float32 2.433 2.433 ... 0.6 0.5667
    clim_dur     (window, z2000_l, yh, xh) float32 61.17 61.77 ... 64.97 67.97
    clim_Im      (window, z2000_l, yh, xh) float32 19.67 20.02 ... 3.457 3.24
    clim_Atot    (window, z2000_l, yh, xh) float32 1.615e+03 1.651e+03 ... 654.5
    clim_HSpeak  (window, z2000_l, yh, xh) float32 21.38 21.72 ... 3.632 3.396

In [18]:
nwindow = 34

# Suppose your final output domain has shape (16, 1080, 1440)
w_size, z_size, y_size, x_size = (nwindow, 16, 1080, 1440)

# Choose chunk sizes for dask
w_chunks, z_chunks, y_chunks, x_chunks = (nwindow, 16, 20, 20)

# Build a dask array that spans the full domain
arr_shape = (w_size, z_size, y_size, x_size)
arr_chunks = (w_chunks, z_chunks, y_chunks, x_chunks)
dask_data  = da.zeros(arr_shape, chunks=arr_chunks, dtype='float32')

# Create a template with that full shape but chunked as you prefer
template = xr.Dataset(
    {
       'clim_freq':   (('window','z2000_l','yh','xh'), dask_data),
       'clim_dur':    (('window','z2000_l','yh','xh'), dask_data),
       'clim_Im':     (('window','z2000_l','yh','xh'), dask_data),
       'clim_Atot':   (('window','z2000_l','yh','xh'), dask_data),
       'clim_HSpeak': (('window','z2000_l','yh','xh'), dask_data),
    },
    coords={
        'window':   test_ds.window,  # length 37
        'z2000_l':  ds.z2000_l, # length 16
        'yh':       ds.yh,      # length 1080
        'xh':       ds.xh,      # length 1440
    }
)

In [19]:
%%time
result = xr.map_blocks(
    process_chunk,            # your function
    ds.thetao,               # the dataset to split into blocks
    template=template,       # structure of each block's output
)

CPU times: user 9min 11s, sys: 7.53 s, total: 9min 19s
Wall time: 9min 15s


In [20]:
%%time
result

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 4.29 µs


<xarray.Dataset>
Dimensions:      (window: 34, z2000_l: 16, yh: 1080, xh: 1440)
Coordinates:
  * window       (window) int64 0 1 2 3 4 5 6 7 8 ... 25 26 27 28 29 30 31 32 33
  * z2000_l      (z2000_l) float64 2.5 10.0 20.0 32.5 ... 500.0 600.0 700.0
  * yh           (yh) float64 -80.39 -80.31 -80.23 -80.15 ... 89.73 89.84 89.95
  * xh           (xh) float64 -299.7 -299.5 -299.2 -299.0 ... 59.53 59.78 60.03
Data variables:
    clim_freq    (window, z2000_l, yh, xh) float32 dask.array<chunksize=(34, 16, 20, 20), meta=np.ndarray>
    clim_dur     (window, z2000_l, yh, xh) float32 dask.array<chunksize=(34, 16, 20, 20), meta=np.ndarray>
    clim_Im      (window, z2000_l, yh, xh) float32 dask.array<chunksize=(34, 16, 20, 20), meta=np.ndarray>
    clim_Atot    (window, z2000_l, yh, xh) float32 dask.array<chunksize=(34, 16, 20, 20), meta=np.ndarray>
    clim_HSpeak  (window, z2000_l, yh, xh) float32 dask.array<chunksize=(34, 16, 20, 20), meta=np.ndarray>

In [ ]:
%%time
result = result.compute()

/nbhome/John.Krasting/envs/py311/lib/python3.11/site-packages/distributed/client.py:3163: UserWarning: Sending large graph of size 693.38 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


In [ ]:
result.to_netcdf("results_window_34.nc")

In [23]:
!ls -lhrt

total 49G
drwxr-xr-x 2 John.Krasting o 4.0K Apr  6 10:08 fortran
-rw-r--r-- 1 John.Krasting o 417K Apr  6 10:09 fast_percentile.cpp
-rwxr-xr-x 1 John.Krasting o  90K Apr  6 10:09 fast_percentile.cpython-311-x86_64-linux-gnu.so
drwxr-xr-x 2 John.Krasting o 4.0K Apr  6 10:09 build
-rw-r--r-- 1 John.Krasting o 475M Apr  6 15:45 results.nc
-rw-r--r-- 1 John.Krasting o   67 Apr  7 09:59 README.md
-rw-r--r-- 1 John.Krasting o  11K Apr  7 10:03 annualcycle.py
-rw-r--r-- 1 John.Krasting o 2.9K Apr  7 10:03 events.py
-rw-r--r-- 1 John.Krasting o 3.9K Apr  7 10:03 fast_percentile.pyx
-rw-r--r-- 1 John.Krasting o 1.8K Apr  7 10:03 mhwmetrics.py
-rw-r--r-- 1 John.Krasting o  327 Apr  7 10:03 setup.py
-rw-r--r-- 1 John.Krasting o 1.1K Apr  7 10:03 util.py
drwxr-xr-x 2 John.Krasting o 4.0K Apr  7 10:11 __pycache__
-rw-r--r-- 1 John.Krasting o  15K Apr  7 12:06 slurm-44417504.out
-rw-r--r-- 1 John.Krasting o 950M Apr  7 12:40 results_window.nc
-rw-r--r-- 1 John.Krasting o  38K Apr  7 15:26 slurm-4441